# 【課題】量子振幅推定

In [ ]:
import sys
import shutil
import tarfile
from google.colab import drive
drive.mount('/content/gdrive')
shutil.copy('/content/gdrive/MyDrive/qcintro.tar.gz', '.')
with tarfile.open('qcintro.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/root/.local')

sys.path.append('/root/.local/lib/python3.10/site-packages')

In [ ]:
# Tested with python 3.10.11, qiskit 0.42.1, numpy 1.23.5, scipy 1.9.3
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Math

# Qiskit関連のパッケージをインポート
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator

# ワークブック独自のモジュール
from qc_workbook.show_state import statevector_expr

print('notebook ready')

In [ ]:
n_state = 3

##################
### EDIT BELOW ###
##################

# state_prepの回路を書いてください
#state_prep = ...

##################
### ABOVE BELOW ###
##################
state_prep.name = "State_Prep"

state_prep.draw('mpl')

In [ ]:
simulator = AerSimulator(method='statevector')

def get_statevector_array(circuit):
    # 渡された回路のコピーを使う
    circuit = circuit.copy()
    # 量子回路の終状態の状態ベクトルを保存するインストラクション
    circuit.save_statevector()
    # 再び「おまじない」のtranspileをしてから、run()に渡す
    circuit = transpile(circuit, backend=simulator)
    job = simulator.run(circuit)
    result = job.result()
    qiskit_statevector = result.data()['statevector']

    # result.data()['statevector']は通常の配列オブジェクト（ndarray）ではなくqiskit独自のクラスのインスタンス
    # ただし np.asarray() で numpy の ndarray に変換可能
    return np.asarray(qiskit_statevector)

statevector = get_statevector_array(state_prep)
expr = statevector_expr(statevector)
Math(expr)

In [ ]:
##################
### EDIT BELOW ###
##################

# Groberの反復回路eを書いてください
# grover_iter = ...?

##################
### ABOVE BELOW ###
##################

grover_iter.decompose().draw('mpl')

In [34]:
n_readout = 4

# 読み出しレジスタ
qreg_readout = QuantumRegister(n_readout, name='readout')
# 状態レジスタ
qreg_state = QuantumRegister(n_state, name='state')
# 読み出し結果が書き出される古典レジスタ
creg_readout = ClassicalRegister(n_readout, name='out')

# 2つの量子レジスタと1つの古典レジスタから量子回路を作る
qc = QuantumCircuit(qreg_readout, qreg_state, creg_readout)

# それぞれのレジスタを初期化
qc.h(qreg_readout)
qc.barrier()

# 状態準備の回路state_prepを固有ベクトルを保持するレジスタに入れる
qc.append(state_prep, qargs = qreg_state)
qc.barrier()

##################
### EDIT BELOW ###
##################

# 読み出しレジスタを制御ビットとして、制御Gゲートを状態レジスタに適用する回路を書いてください。

##################
### ABOVE BELOW ###
##################

In [ ]:
def qft_dagger(qreg):
    """逆量子フーリエ変換用の回路"""
    qc = QuantumCircuit(qreg)

    for j in range(qreg.size // 2):
        qc.swap(qreg[j], qreg[-1 - j])

    for itarg in range(qreg.size):
        for ictrl in range(itarg):
            power = ictrl - itarg - 1
            qc.cp(-2. * np.pi * (2 ** power), ictrl, itarg)

        qc.h(itarg)

    qc.name = "QFT^dagger"
    return qc

qc.barrier()
# 読み出しレジスタに逆フーリエ変換の回路を追加
qc.append(qft_dagger(qreg_readout), qargs = qreg_readout)
qc.barrier()
qc.measure(qreg_readout, creg_readout)
#qc.decompose().draw('mpl')
qc.draw('mpl')

In [ ]:
from qiskit.primitives import Sampler
sampler = Sampler()

# Now run the job and examine the results
sampler_job = sampler.run(qc)
result = sampler_job.result()

from qiskit.visualization import plot_distribution
plt.style.use('dark_background')
plot_distribution(result.quasi_dists[0])

**提出するもの**

- 以下を行う回路
  - 状態を準備する
  - グローバーの反復Gを行う
  - 読み出しレジスタを制御ビットとして、制御Gゲートを状態レジスタに適用する
- 量子振幅推定を行った結果のヒストグラムと、その解釈

**量子振幅推定の結果の解釈**
